# SimpleMetrics的API查询

上一节我们对不同类型的Metrics进行了举例说明，这一节将列出**PerResidueRealMetrics, RealMetric，StringMetric， 和CompositeRealMetrics** 所有API的使用方式。

In [1]:
#加载与SimpleMetrics调用有关的class.
from pyrosetta import pose_from_pdb, init
from pyrosetta.rosetta.core.simple_metrics.metrics import *
from pyrosetta.rosetta.core.simple_metrics.per_residue_metrics import *
from pyrosetta.rosetta.core.simple_metrics.composite_metrics import *
from pyrosetta.rosetta.core.simple_metrics import *
init()
pose = pose_from_pdb('./data/6LZ9_H_L.pdb')
ref_pose = pose_from_pdb('./data/3K2U_H_L.pdb')

PyRosetta-4 2021 [Rosetta PyRosetta4.conda.mac.cxx11thread.serialization.python37.Release 2021.31+release.c7009b3115c22daa9efe2805d9d1ebba08426a54 2021-08-07T10:04:12] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.cxx11thread.serialization.python37.Release r292 2021.31+release.c7009b3115c c7009b3115c22daa9efe2805d9d1ebba08426a54 http://www.pyrosetta.org 2021-08-07T10:04:12
core.init: {0} command: PyRosetta -ex1 -ex2aro -database /opt/miniconda3/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=2119001967 seed_offset=0 real_seed=2119001967 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=2119001967 RG_type=mt19937
core.chemical.GlobalResid

## 1.  PerResidueRealMetrics

### 1.1 HbondMetric
计算一个selector或者两个selector间每个残基的氢键数量。

In [2]:
#example1 计算一个selector上每个残基的氢键数量
from pyrosetta.rosetta.core.select.residue_selector import ResidueIndexSelector
hbond = HbondMetric()
hbond_selector = ResidueIndexSelector('67H-71H')
hbond.set_residue_selector(hbond_selector)
hbond.apply(pose,'one_selector_')

basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBPoly1D.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBFadeIntervals.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/HBEval.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/DonStrength.csv
basic.io.database: {0} Database file opened: scoring/score_functions/hbonds/ref2015_params/AccStrength.csv


In [3]:
from pyrosetta.rosetta.core.simple_metrics import get_sm_data
sm_data = get_sm_data(pose)
per_real_metric = sm_data.get_per_residue_real_metric_data()
per_real_metric['one_selector_hbonds']

map_unsigned_long_double{66: 1, 67: 2, 68: 0, 69: 2, 70: 2}

In [4]:
#example2 计算两个selector间每个残基的氢键数量
hbond = HbondMetric()
hbond_selector = ResidueIndexSelector('67H-71H')
hbond_selector2 = ResidueIndexSelector('78H-82H')
hbond.set_residue_selector(hbond_selector)
hbond.set_residue_selector2(hbond_selector2)
hbond.apply(pose,'two_selector_')

In [5]:
from pyrosetta.rosetta.core.simple_metrics import get_sm_data
sm_data = get_sm_data(pose)
per_real_metric = sm_data.get_per_residue_real_metric_data()
per_real_metric['two_selector_hbonds']

map_unsigned_long_double{66: 0, 67: 2, 68: 0, 69: 2, 70: 0}

### 1.2 WaterMediatedHbondMetric
分析选择残基水介导的氢键情况。

In [6]:
from pyrosetta.rosetta.core.select.residue_selector import TrueResidueSelector
init('-ignore_waters false') # 初始化时不能把水自动处理掉了！
wmh_pose = pose_from_pdb('./data/4r80.pdb')
wmh_selector = TrueResidueSelector()
wm_hbond = WaterMediatedHbondMetric()
wm_hbond.set_residue_selector(wmh_selector)
wm_hbond.set_depth(1) #介导氢键的水分子数量，默认是1个水分子
wm_hbond.set_include_only_set_depth(False) #是否仅包括指定的depth。True包括指定的depth个水分子,False包括1-depth个水分子。
wm_hbond.apply(pose)

PyRosetta-4 2021 [Rosetta PyRosetta4.conda.mac.cxx11thread.serialization.python37.Release 2021.31+release.c7009b3115c22daa9efe2805d9d1ebba08426a54 2021-08-07T10:04:12] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.cxx11thread.serialization.python37.Release r292 2021.31+release.c7009b3115c c7009b3115c22daa9efe2805d9d1ebba08426a54 http://www.pyrosetta.org 2021-08-07T10:04:12
core.init: {0} command: PyRosetta -ignore_waters false -database /opt/miniconda3/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=-1568603508 seed_offset=0 real_seed=-1568603508 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=-1568603508 RG_type=mt19937
core.import_po

In [7]:
# 没有显示水介导的氢键
per_real_metric = sm_data.get_per_residue_real_metric_data()
per_real_metric['water_mediated_hbonds']

map_unsigned_long_double{1: 0, 2: 0, 3: 0, 4: 0, 5: 0, 6: 0, 7: 0, 8: 0, 9: 0, 10: 0, 11: 0, 12: 0, 13: 0, 14: 0, 15: 0, 16: 0, 17: 0, 18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 0, 24: 0, 25: 0, 26: 0, 27: 0, 28: 0, 29: 0, 30: 0, 31: 0, 32: 0, 33: 0, 34: 0, 35: 0, 36: 0, 37: 0, 38: 0, 39: 0, 40: 0, 41: 0, 42: 0, 43: 0, 44: 0, 45: 0, 46: 0, 47: 0, 48: 0, 49: 0, 50: 0, 51: 0, 52: 0, 53: 0, 54: 0, 55: 0, 56: 0, 57: 0, 58: 0, 59: 0, 60: 0, 61: 0, 62: 0, 63: 0, 64: 0, 65: 0, 66: 0, 67: 0, 68: 0, 69: 0, 70: 0, 71: 0, 72: 0, 73: 0, 74: 0, 75: 0, 76: 0, 77: 0, 78: 0, 79: 0, 80: 0, 81: 0, 82: 0, 83: 0, 84: 0, 85: 0, 86: 0, 87: 0, 88: 0, 89: 0, 90: 0, 91: 0, 92: 0, 93: 0, 94: 0, 95: 0, 96: 0, 97: 0, 98: 0, 99: 0, 100: 0, 101: 0, 102: 0, 103: 0, 104: 0, 105: 0, 106: 0, 107: 0, 108: 0, 109: 0, 110: 0, 111: 0, 112: 0, 113: 0, 114: 0, 115: 0, 116: 0, 117: 0, 118: 0, 119: 0, 120: 0, 121: 0, 122: 0, 123: 0, 124: 0, 125: 0, 126: 0, 127: 0, 128: 0, 129: 0, 130: 0, 131: 0, 132: 0, 133: 0, 134: 0, 135: 0, 136

[bug] 此Metrics截止在2021.07之前，有问题，不会输出数量。

### 1.3 PerResidueClashMetric
通过每个原子的兰纳-琼斯半径计算两个selector之间原子冲突数量，默认使用软核,LJ势减少33%。

In [8]:
from pyrosetta.rosetta.core.simple_metrics.per_residue_metrics import *
inner_selector = ResidueIndexSelector('18-24')
outer_selector = ResidueIndexSelector('25-30')
clash_num = PerResidueClashMetric(inner_selector,outer_selector)
clash_num.set_use_soft_clash(True) # True代表根据原子间距离小于(atomI_LJ + atomJ_LJ)*(1 - dampening_percent)判断冲突。
clash_num.set_soft_dampening(0.01) # 设置dampening_percent 默认0.33
clash_num.set_use_hydrogens(False) #True为默认，仅通过重原子判断冲突，False通过氢原子判断冲突 
clash_num.apply(pose,'per_resid_')

core.scoring.etable: {0} Starting energy table calculation
core.scoring.etable: {0} smooth_etable: changing atr/rep split to bottom of energy well
core.scoring.etable: {0} smooth_etable: spline smoothing lj etables (maxdis = 6)
core.scoring.etable: {0} smooth_etable: spline smoothing solvation etables (max_dis = 6)
core.scoring.etable: {0} Finished calculating energy tables.
core.simple_metrics.per_residue_metrics.PerResidueClashMetric: {0} [ WARNING ] ################ Cloning pose and Scoring! ##############################
core.simple_metrics.per_residue_metrics.PerResidueClashMetric: {0} [ WARNING ] Ensure that pose is scored
core.simple_metrics.per_residue_metrics.PerResidueClashMetric: {0} [ WARNING ] before PerResidueClashMetric for maximum performance!
core.simple_metrics.per_residue_metrics.PerResidueClashMetric: {0} [ WARNING ] ##########################################################################
core.simple_metrics.per_residue_metrics.PerResidueClashMetric: {0} Bonded:  

In [9]:
per_real_metric = sm_data.get_per_residue_real_metric_data()
per_real_metric['per_resid_atomic_clashes']

map_unsigned_long_double{18: 0, 19: 0, 20: 0, 21: 0, 22: 0, 23: 2, 24: 7}

### 1.4 PerResidueRMSDMetric
计算两个selector中两段指定残基的每个残基的RMSD值。主selector与ref_selector中原子数目必须相等。<br />
type: [rmsd_all, rmsd_all_heavy, rmsd_protein_bb_ca, rmsd_protein_bb_heavy, rmsd_protein_bb_heavy_including_O, rmsd_sc, rmsd_sc_heavy]

In [10]:
#计算两个selector的两段指定残基的RMSD
from pyrosetta.rosetta.core.simple_metrics.per_residue_metrics import *
from pyrosetta.rosetta.core.scoring import rmsd_atoms
rmsd_type = rmsd_atoms.rmsd_protein_bb_ca
per_residue_rmsd = PerResidueRMSDMetric()
range_selector = ResidueIndexSelector('46H-50H')
range_selector2 = ResidueIndexSelector('62L-66L')
per_residue_rmsd.set_residue_selector(range_selector) #只计算子集
per_residue_rmsd.set_residue_selector_reference(range_selector2) #只计算子集
per_residue_rmsd.set_comparison_pose(ref_pose)
per_residue_rmsd.set_rmsd_type(rmsd_atoms.rmsd_protein_bb_ca) #CA only
per_residue_rmsd.set_desymmetrize_residue_selector(True) #是否去对称化，默认为True
per_residue_rmsd.apply(pose,'per_')

In [11]:
from pyrosetta.rosetta.core.simple_metrics import get_sm_data
sm_data = get_sm_data(pose)
per_real_metric = sm_data.get_per_residue_real_metric_data()
per_real_metric['per_res_rmsd']

map_unsigned_long_double{45: 114.144, 46: 113.949, 47: 113.95, 48: 114.177, 49: 110.526}

### 1.5 PerResidueDensityFitMetric
计算一个模型与加载的电子密度的相关性或者Z-score.

In [12]:
#fit_density = PerResidueDensityFitMetric()
#fit_density.set_match_mode(0) # 0=zscore 1=density correlation 默认0
#fit_density.set_residue_selector(range_selector) #选取部分进行计算
#fit_density.set_sliding_window_size(3)
#fit_density.set_mixed_sliding_window(1) #设置混合窗口 3 for protein, 1 for everything else。可能使得结果有偏向性。
#fit_density.apply(pose,'prefix')

## 2. RealMetrics

### 2.1 RMSDMetric
计算两个selector组分之间的RMSD值。 目前支持的type有
- rmsd_all
- rmsd_all_heavy
- rmsd_protein_bb_ca
- rmsd_protein_bb_heavy
- rmsd_protein_bb_heavy_including_O
- rmsd_sc
- rmsd_sc_heavy

默认为rmsd_all

<center><img src="./img/6LZ9_rmsd.png" width = "400" height = "300" align=center /> </center>
(图片来源: 晶泰科技团队)

In [13]:
#计算图中两个selector主链CA的RMSD
from pyrosetta.rosetta.core.select.residue_selector import ResidueIndexSelector
from pyrosetta.rosetta.core.scoring import rmsd_atoms
range_selector = ResidueIndexSelector('46H-50H')
range_selector2 = ResidueIndexSelector('62L-66L')
rmsd = RMSDMetric(pose)
rmsd.set_residue_selector(range_selector)
rmsd.set_residue_selector_reference(range_selector2)
rmsd.set_rmsd_type(rmsd_atoms.rmsd_protein_bb_ca) #指定具体的某项RMSD.
rmsd.apply(pose,'two_sel_')

In [14]:
from pyrosetta.rosetta.core.simple_metrics import get_sm_data
sm_data = get_sm_data(pose)
real_metric = sm_data.get_real_metric_data()
real_metric['two_sel_rmsd']

25.169509903849935

### 2.2 TotalEnergyMetric
计算总能量的变化，或者计算系统的总能。 如果设置了set_comparison_pose，那么就会计算两个pose指定残基的总能量差。

In [15]:
# 计算两个pose的H链的能量差
from pyrosetta.rosetta.core.select.residue_selector import ResidueIndexSelector, ChainSelector
from pyrosetta import create_score_function
select_heavy_chain = ChainSelector('H')
total_energy = TotalEnergyMetric()
total_energy.set_residue_selector(select_heavy_chain)
scorefxn = create_score_function('ref2015')
total_energy.set_scorefunction(scorefxn)
total_energy.set_comparison_pose(ref_pose) #不指定则只计算pose的总能量
total_energy.apply(pose,'two_pose_H_')

basic.io.database: {0} Database file opened: scoring/score_functions/rama/fd/all.ramaProb
basic.io.database: {0} Database file opened: scoring/score_functions/rama/fd/prepro.ramaProb
basic.io.database: {0} Database file opened: scoring/score_functions/omega/omega_ppdep.all.txt
basic.io.database: {0} Database file opened: scoring/score_functions/omega/omega_ppdep.gly.txt
basic.io.database: {0} Database file opened: scoring/score_functions/omega/omega_ppdep.pro.txt
basic.io.database: {0} Database file opened: scoring/score_functions/omega/omega_ppdep.valile.txt
basic.io.database: {0} Database file opened: scoring/score_functions/P_AA_pp/P_AA
basic.io.database: {0} Database file opened: scoring/score_functions/P_AA_pp/P_AA_n
core.scoring.P_AA: {0} shapovalov_lib::shap_p_aa_pp_smooth_level of 1( aka low_smooth ) got activated.
basic.io.database: {0} Database file opened: scoring/score_functions/P_AA_pp/shapovalov/10deg/kappa131/a20.prop
basic.io.database: {0} Database file opened: scoring/

In [16]:
real_metric = sm_data.get_real_metric_data()
real_metric['two_pose_H_total_energy']

98.51662779368385

### 2.3 SequenceSimilarityMetric
计算pose和参考序列之间的相似性。根据BLOSUM62矩阵计算每个位置。 如果normalize设置了，那么将BLOSUM62 scores/position num.进行标准化。

In [17]:
#选择内核层残基(SASA法)
from pyrosetta.rosetta.core.select.residue_selector import LayerSelector
layer_selector = LayerSelector()
layer_selector.set_layers(True, False, False) # 只选择内核的设置方法
layer_selector.set_use_sc_neighbors(False) # 使用Sidechain neighbor算法，否则使用SASA-based算法，确定内核残基。
layer_selector.set_cutoffs(20.0, 40.0) # 内核,表层截断半径设置
layer_selector.set_ball_radius(2.0) # 传统值为1.4

core.select.residue_selector.LayerSelector: {0} Setting LayerSelector to use sidechain neighbors to determine burial.
core.select.residue_selector.LayerSelector: {0} Set cutoffs for core and surface to 5.2 and 2, respectively, in LayerSelector.
core.select.residue_selector.LayerSelector: {0} Setting core=true boundary=false surface=false in LayerSelector.
core.select.residue_selector.LayerSelector: {0} Setting LayerSelector to use rolling ball-based occlusion to determine burial.
core.select.residue_selector.LayerSelector: {0} Set cutoffs for core and surface to 20 and 40, respectively, in LayerSelector.
core.select.residue_selector.LayerSelector: {0} Setting radius for rolling ball algorithm to 2 in LayerSelector.  (Note that this will have no effect if the sidechain neighbors method is used.)


In [18]:
seq_similar = SequenceSimilarityMetric()
seq_similar.set_residue_selector(layer_selector)
seq_similar.set_native_pose(ref_pose)
seq_similar.set_apply_selector_to_native(True) #是否将选择的残基用于native_pose 默认：False
seq_similar.set_normalize(True) # 是否标准化处理，默认为True(即：标准化处理)
seq_similar.apply(pose,'core_layer_')

In [19]:
real_metric = sm_data.get_real_metric_data()
real_metric['core_layer_sequence_similarity']

-1.2407407407407407

### 2.4 SequenceRecoveryMetric

计算两个pose的序列上与原序列的恢复率。有4种比对模式: 
- Standard: set_comparison_pose() 
- Pass/Fail: load_pssm 
- PSSM/Ave: load_pssm、use_ave_pssm() 
- PSSM/Delta load_pssm、use_ave_pssm()、set_comparison_pose()

In [20]:
from pyrosetta.rosetta.protocols.analysis.simple_metrics import *
seq_recovery = SequenceRecoveryMetric()
seq_recovery.set_comparison_pose(ref_pose)
seq_recovery_selector = ResidueIndexSelector('5H-30H')
seq_recovery_selector_ref = ResidueIndexSelector('55H-80H')
seq_recovery.set_residue_selector(seq_recovery_selector)
seq_recovery.set_residue_selector_ref(seq_recovery_selector_ref)
#seq_recovery.load_pssm(pssm_file) #额外设置
seq_recovery.set_use_ave_pssm(True) #是否使用ave_pssm
seq_recovery.apply(pose,'two_pose_')

protocols.analysis.simple_metrics.SequenceRecoveryMetric: {0} Calculating sequence metric using the Standard settings.


In [21]:
real_metric = sm_data.get_real_metric_data()
real_metric['two_pose_seqrec']

0.038461538461538464

### 2.5  DihedralDistanceMetric
计算两个pose或一个pose中两个区域的二面角之间的标准化距离。(degrees为单位)，可以理解为构象相似性大小。

In [22]:
#example1 计算一个pose中两个区域的二面角之间的标准化距离
from pyrosetta.rosetta.core.select.residue_selector import ResidueIndexSelector
dd_selector = ResidueIndexSelector('18H-24H')
dd_selector2 = ResidueIndexSelector('58H-64H')
dihedral_distance = DihedralDistanceMetric()
dihedral_distance.set_comparison_pose(pose)
dihedral_distance.set_include_protein_omega(False) #是否包含omega角
dihedral_distance.set_residue_selector(dd_selector)
dihedral_distance.set_residue_selector_reference(dd_selector2)
dihedral_distance.apply(pose, 'one_pose_')

In [23]:
from pyrosetta.rosetta.core.simple_metrics import get_sm_data
sm_data = get_sm_data(pose)
real_metric = sm_data.get_real_metric_data()
real_metric['one_pose_dihedral_distance']

70.54020894906083

In [24]:
#example2 计算两个pose中两个区域的二面角之间的标准化距离
from pyrosetta.rosetta.core.select.residue_selector import ResidueIndexSelector
dd_selector = ResidueIndexSelector('18H-24H')
dd_selector2 = ResidueIndexSelector('58H-64H')
dihedral_distance = DihedralDistanceMetric()
dihedral_distance.set_comparison_pose(ref_pose)
dihedral_distance.set_include_protein_omega(False) #是否包含omega角
dihedral_distance.set_residue_selector(dd_selector)
dihedral_distance.set_residue_selector_reference(dd_selector2)
dihedral_distance.apply(pose, 'two_pose_')

In [25]:
real_metric = sm_data.get_real_metric_data()
real_metric['two_pose_dihedral_distance']

64.26955898015767

**结果解读** <br />
数值越小说明两个区域的残基结构越相似。

### 2.6   InteractionEnergyMetric
计算selector与其余残基或selector与另一个selector之间的长程或短程相互作用的能量。可以设置为只计算短程、长程、或则某一项相互作用的能量。 相互作用选项:/fa_atr/fa_rep/fa_sol/fa_elec/lk_ball_wtd/hbond_bb_sc/hbond_sc，此外rama_prepro和proclose能量项可选。

In [26]:
range_selector3 = ResidueIndexSelector('25H-30H')
residue_selector3 = range_selector3.apply(pose)
print(residue_selector3)

vector1_bool[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [27]:
#set_ignore_scoretypes传参需要一个vector1,因此我们需要额外创建一个vector1_core_scoring_ScoreType;
from pyrosetta.rosetta.utility import vector1_core_scoring_ScoreType
from pyrosetta.rosetta.core.scoring import fa_atr, fa_rep, fa_elec
from pyrosetta import create_score_function
ie_selector = ResidueIndexSelector('18H-24H')
ie_selector2 = ResidueIndexSelector('25H-30H')

vector1 = vector1_core_scoring_ScoreType()
vector1.append(fa_atr)
interaction_energy = InteractionEnergyMetric(ie_selector, ie_selector2)
scorefxn = create_score_function('ref2015') #默认ref2015
interaction_energy.set_scorefunction(scorefxn) 
interaction_energy.set_include_rama_prepro_and_proclose(False) #是否包括rama_prepro和proclose能量项
interaction_energy.set_ignore_scoretypes(vector1) #用于忽略某项能量项
#interaction_energy.set_include_only_scoretypes(vector1) #只包括某项能量，但有些问题！！
interaction_energy.apply(pose, 'two_selectors_')

core.scoring.ScoreFunctionFactory: {0} SCOREFUNCTION: ref2015
core.simple_metrics.metrics.InteractionEnergyMetric: {0} [ WARNING ] ################ Cloning pose and Scoring! ##############################
core.simple_metrics.metrics.InteractionEnergyMetric: {0} [ WARNING ] Ensure that pose is scored
core.simple_metrics.metrics.InteractionEnergyMetric: {0} [ WARNING ] before using InteractionEnergyMetric for maximum performance!
core.simple_metrics.metrics.InteractionEnergyMetric: {0} [ WARNING ] ##########################################################################
core.simple_metrics.metrics.InteractionEnergyMetric: {0} 
core.simple_metrics.metrics.InteractionEnergyMetric: {0} fa_rep 0.55
core.simple_metrics.metrics.InteractionEnergyMetric: {0} Unweighted: 3.05322
core.simple_metrics.metrics.InteractionEnergyMetric: {0} Weighted: 1.67927
core.simple_metrics.metrics.InteractionEnergyMetric: {0} 
core.simple_metrics.metrics.InteractionEnergyMetric: {0} fa_sol 1
core.simple_metrics.m

In [28]:
real_metric = sm_data.get_real_metric_data()
real_metric['two_selectors_interaction_energy']

0.34420423407252454

## 3. StringMetrics

### 3.1 SequenceMetric
返回序列的氨基酸缩写。 Mode= oneletter, threeletter, basename, or fullname.

In [29]:
seq_AA_selector = ResidueIndexSelector('1L-20L')
seq_AA = SequenceMetric(seq_AA_selector)
seq_AA.set_output_mode('fullname')
prefix = 'seq_AA_'
seq_AA.apply(pose, prefix)

In [30]:
from pyrosetta.rosetta.core.simple_metrics import get_sm_data
sm_data = get_sm_data(pose)
string_metric = sm_data.get_string_metric_data()
string_metric['seq_AA_sequence']

'ASP:NtermProteinFull,ILE,GLN,MET,THR,GLN,THR,THR,SER,SER,LEU,SER,ALA,SER,LEU,GLY,ASP,ARG,VAL,THR,'

### 3.2 SecondaryStructureMetric
返回每个氨基酸的DSSP二级结构的定义，H:螺旋 E:折叠 L:loop

In [31]:
select_heavy_chain = ChainSelector('H')
dssp = SecondaryStructureMetric(select_heavy_chain)
dssp.set_use_dssp_reduced(True) #是否使用简化的DSSP字母，默认为True
dssp.apply(pose,'H_')

In [32]:
from pyrosetta.rosetta.core.simple_metrics import get_sm_data
sm_data = get_sm_data(pose)
string_metric = sm_data.get_string_metric_data()
string_metric['H_secondary_structure']

'LEEEELLLLELLLLLLEEEEEEEELLLHHHLLEEEEEELLLLLLEEEEEELLLLLEEELLLLHHHEEEEEELLLLEEEEEELLLLHHHLEEEEEEELLHHHHLLLLLLEELLLEEEEL'

### 3.3 ResidueSummaryMetric
ResidueSummaryMetric是基于PerResidueSasaMetric计算结果进行数据处理，具体的方式如下:<br />
mean, n_res_eq, n_res_gt, n_res_gt_or_eq, n_res_lt, n_res_lt_or_eq, n_res_ne, sum。

In [33]:
resi_sum = ResidueSummaryMetric()
resi_sum.set_action(mean)
resi_sum.set_action_value(0) #基于一个值计算N个残基某个Metrics,默认是0
resi_sum.set_epsilon(0.0001) #指定有效数字，尤其在action = n_res_eq or n_res_ne时，两者差的绝对值小于这个有效数字认为相等
resi_sum.set_fail_on_missing_cache(True) #如果设置了use_cached_data但cache没有找到，此时是否fail
resi_sum.set_metric(per_residue_rmsd) 
resi_sum.apply(pose,'pose_')

core.simple_metrics.metrics.ResidueSummaryMetric: {0} Returning the mean of the data


In [34]:
from pyrosetta.rosetta.core.simple_metrics import get_sm_data
sm_data = get_sm_data(pose)
real_metric = sm_data.get_real_metric_data()
real_metric['pose_res_summary']

113.34897941753817

### 3.4 InteractionGraphSummaryMetric
在第6章Rosetta Packer这一节已经讲过InteractionGraph。InteractionGraphSummaryMetric用于Task操作中，InteractionGraph的可读化，用于优化Rosetta的模拟退火。

In [35]:
# 先设置task
from pyrosetta.rosetta.core.pack.task import TaskFactory
from pyrosetta.rosetta.core.pack.task.operation import InitializeFromCommandline, RestrictToRepacking
pack_task = TaskFactory()
pack_task.push_back(InitializeFromCommandline())
pack_task.push_back(RestrictToRepacking())
# 引入需要计算的pose
demo_pose = pose_from_pdb('./data/pose_demo.pdb')

core.import_pose.import_pose: {0} File './data/pose_demo.pdb' automatically determined to be of type PDB
core.conformation.Conformation: {0} [ WARNING ] missing heavyatom:  OXT on residue SER:CtermProteinFull 12
core.conformation.Conformation: {0} Found disulfide between residues 2 11
core.conformation.Conformation: {0} current variant for 2 CYS
core.conformation.Conformation: {0} current variant for 11 CYS
core.conformation.Conformation: {0} current variant for 2 CYD
core.conformation.Conformation: {0} current variant for 11 CYD


<center><img src="./img/InteractionGraph.png" width = "400" height = "300" align=center /> </center>
(图片来源: 晶泰科技团队)

In [36]:
from pyrosetta.rosetta.protocols.quantum_annealing import *
interaction_graph = InteractionGraphSummaryMetric()
scorefxn = create_score_function('ref2015') 
interaction_graph.set_scorefunction(scorefxn)
interaction_graph.set_short_version(True) #是否简短的输出结果
interaction_graph.set_task_factory(pack_task)
interaction_graph.apply(demo_pose,'prefix')

core.pack.task: {0} Packer task: initialize from command line()
core.pack.pack_rotamers: {0} built 144 rotamers at 12 positions.
core.pack.pack_rotamers: {0} Requesting all available threads for interaction graph computation.
core.pack.interaction_graph.interaction_graph_factory: {0} Instantiating DensePDInteractionGraph
basic.thread_manager.RosettaThreadManager: {?} Creating a thread pool of 1 threads.
basic.thread_manager.RosettaThreadPool: {?} Launched 0 new threads.
core.pack.rotamer_set.RotamerSets: {0} Completed interaction graph pre-calculation in 1 available threads (1 had been requested).


In [37]:
from pyrosetta.rosetta.core.simple_metrics import get_sm_data
sm_data = get_sm_data(demo_pose)
string_metric = sm_data.get_string_metric_data()
# print(string_metric) 输出很长，需要时在启用!!!

**结果解读** <br />
[BEGIN ONEBODY SEQPOS/ROTINDEX/ENERGY] 这里ONEBODY指单个残基的Rotamer，SEQPOS/ROTINDEX/ENERGY对应的是残基序列、Rotamer序列和相应的能量。<br />
[BEGIN TWOBODY SEQPOS1/ROTINDEX1/SEQPOS2/ROTINDEX2/ENERGY] 这里TWOBODY指两个残基的Rotamer组合，SEQPOS1/ROTINDEX1/
SEQPOS2/ROTINDEX2/ENERGY对应的是残基序列1、残基序列1的Rotamer序列、残基序列2、残基序列2的Rotamer序列和这个组合的能量。

### 3.5 PolarGroupBurialPyMolStringMetric
返回Pymol极性区域的着色命令.根据buried_unsatisfied_penalty打分并进行着色.

In [41]:
from pyrosetta.rosetta.protocols.analysis.burial_metrics import *
init('-mute all') # 输出很长,mute省掉了吧
h_pose = pose_from_pdb('./data/6LZ9_H.pdb')
polar_metric = PolarGroupBurialPyMolStringMetric()
polar_metric.set_scorefxn(scorefxn)
polar_metric.set_verbose(True)
prefix = 'pgb_'
polar_metric.apply(h_pose, prefix)

PyRosetta-4 2021 [Rosetta PyRosetta4.conda.mac.cxx11thread.serialization.python37.Release 2021.31+release.c7009b3115c22daa9efe2805d9d1ebba08426a54 2021-08-07T10:04:12] retrieved from: http://www.pyrosetta.org
(C) Copyright Rosetta Commons Member Institutions. Created in JHU by Sergey Lyskov and PyRosetta Team.
core.init: {0} Checking for fconfig files in pwd and ./rosetta/flags
core.init: {0} Rosetta version: PyRosetta4.conda.mac.cxx11thread.serialization.python37.Release r292 2021.31+release.c7009b3115c c7009b3115c22daa9efe2805d9d1ebba08426a54 http://www.pyrosetta.org 2021-08-07T10:04:12
core.init: {0} command: PyRosetta -mute all -database /opt/miniconda3/lib/python3.7/site-packages/pyrosetta/database
basic.random.init_random_generator: {0} 'RNG device' seed mode, using '/dev/urandom', seed=-492439020 seed_offset=0 real_seed=-492439020 thread_index=0
basic.random.init_random_generator: {0} RandomGenerator:init: Normal mode, seed=-492439020 RG_type=mt19937
core.import_pose.import_pose

In [39]:
from pyrosetta.rosetta.core.simple_metrics import get_sm_data
sm_data = get_sm_data(h_pose)
string_metric = sm_data.get_string_metric_data()
pymol_cmd = string_metric['pgb_polar_group_burial_pymol']
pymol_cmd.replace("\n",";")

'Pymol commands to colour the pose:;color grey;color cyan, resi 1 AND name O;color cyan, resi 1 AND name N+1H+2H+3H;color cyan, resi 2 AND name O;color cyan, resi 2 AND name OE1;color cyan, resi 2 AND name N+H;color cyan, resi 2 AND name NE2+1HE2+2HE2;color cyan, resi 3 AND name O;color cyan, resi 3 AND name N+H;color cyan, resi 4 AND name O;color cyan, resi 4 AND name N+H;color cyan, resi 4 AND name NZ+1HZ+2HZ+3HZ;color cyan, resi 5 AND name O;color cyan, resi 5 AND name OE1;color cyan, resi 5 AND name OE2;color cyan, resi 5 AND name N+H;color cyan, resi 6 AND name O;color cyan, resi 6 AND name OG+HG;color cyan, resi 6 AND name N+H;color cyan, resi 7 AND name O;color cyan, resi 7 AND name N+H;color cyan, resi 8 AND name O;color cyan, resi 9 AND name O;color cyan, resi 9 AND name OD1;color cyan, resi 9 AND name OD2;color cyan, resi 9 AND name N+H;color cyan, resi 10 AND name O;color cyan, resi 10 AND name N+H;color cyan, resi 11 AND name O;color cyan, resi 11 AND name N+H;color cyan, r

把字符串粘贴到Pymol中时，包埋的非饱和极性原子标注为橙色，暴露的极性原子标记为青色。很容易一眼识别出来！

<center><img src="./img/6LZ9_H_polar.png" width = "400" height = "300" align=center /> </center>
(图片来源: 晶泰科技团队)

## 4. Composite Metrics

### 4.1 ElectrostaticComplementMetric
使用APBS计算McCoy, Chandana, Colman Electrostatic complementarity

**[linux]*注意：需要安装APBS: conda install -c conda-forge apbs*

In [40]:
from pyrosetta.rosetta.core.simple_metrics.composite_metrics import ElectrostaticComplementarityMetric
Electrostatic = ElectrostaticComplementarityMetric()
Electrostatic.ignore_radius(20) #忽略距离界面多远的原子，-1代表保持所有的原子。
Electrostatic.interface_trim_radius(0) # 距离SASA表面多远的分子点截断，默认是0
Electrostatic.partially_solvated(True) # 是否部分溶剂化，True为部分溶剂化，False为全部溶剂化
Electrostatic.report_all_ec(True) #是否输出两个selector的具体数值，否则只输出平均值
EC_selector1 = ResidueIndexSelector('2H-10H')
EC_selector2 = ResidueIndexSelector('11H-20H')
Electrostatic.residue_selector1(EC_selector1)
Electrostatic.residue_selector2(EC_selector2)
Electrostatic.apply(pose,'pose_')

basic.io.database: {0} Database file opened: scoring/score_functions/sc/sc_radii.lib


RuntimeError: 

File: /Volumes/MacintoshHD3/benchmark/W.fujii.release/rosetta.Fujii.release/_commits_/main/source/src/core/scoring/PoissonBoltzmannPotential.cc:306
[ ERROR ] UtilityExitException
ERROR: APBS failed to generate the result file.



In [ ]:
from pyrosetta.rosetta.core.simple_metrics import get_sm_data
sm_data = get_sm_data(pose)
composite_real_metric = sm_data.get_composite_real_metric_data()
composite_real_metric['pose_ec']